# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hami,42.8000,93.4500,55.87,20,2,2.98,CN,1677997839
1,bubaque,11.2833,-15.8333,75.00,68,85,9.95,GW,1677997840
2,kapaa,22.0752,-159.3190,73.72,88,75,13.80,US,1677997840
3,barrow,71.2906,-156.7887,-18.38,69,20,21.85,US,1677997841
4,nikolskoye,59.7035,30.7861,28.80,94,100,1.92,RU,1677997841


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
locations = city_data_df[['Lat','Lng']]

humidity = city_data_df['Humidity']

In [7]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
ideal_city_df = city_data_df.copy()
ideal_city_df = ideal_city_df.loc[(ideal_city_df['Cloudiness'] < 20) 
                                  & (ideal_city_df['Max Temp'] > 65) 
                                  & (ideal_city_df['Max Temp'] < 90)
                                  & (ideal_city_df['Humidity'] < 50)
                                  & (ideal_city_df['Wind Speed'] < 10)]
ideal_city_df.dropna(inplace=True)
ideal_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,gwadar,25.1216,62.3254,80.22,48,0,6.69,PK,1677997843
12,jijiga,9.3500,42.8000,73.13,36,12,7.52,ET,1677997845
17,victoria,22.2855,114.1577,75.25,42,0,1.99,HK,1677997584
69,lalsot,26.5667,76.3333,87.64,23,0,7.09,IN,1677997873
115,hurghada,27.2574,33.8129,73.44,21,0,0.00,EG,1677997725


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = ideal_city_df[['City','Country','Lat','Lng','Humidity']]
hotel_df.reset_index(inplace=True, drop=True)
hotel_df.head()

,City,Country,Lat,Lng,Humidity
0,gwadar,PK,25.1216,62.3254,48
1,jijiga,ET,9.3500,42.8000,36
2,victoria,HK,22.2855,114.1577,42
3,lalsot,IN,26.5667,76.3333,23
4,hurghada,EG,27.2574,33.8129,21


In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_names = []

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    params = {
    "location": f"{row['Lat']},{row['Lng']}",
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}, {row['Country']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    try:
        print(f"Closest hotel is {response['results'][0]['name']}.")
        hotel_names.append(response['results'][0]['name'])
        print("------------")
    except:
        print("No Results Found... Skipping")
        hotel_names.append("N/A")

hotel_df['Hotel Name'] = hotel_names

Retrieving Results for Index 0: gwadar, PK.
Closest hotel is Al-Salam Hotel and Guest House.
------------
Retrieving Results for Index 1: jijiga, ET.
Closest hotel is Hassan Wali Hotel.
------------
Retrieving Results for Index 2: victoria, HK.
Closest hotel is Four Seasons Hotel Hong Kong.
------------
Retrieving Results for Index 3: lalsot, IN.
Closest hotel is Hotel Kesar Gulab.
------------
Retrieving Results for Index 4: hurghada, EG.
Closest hotel is Royal Lagoons Resort & Aqua Park.
------------
Retrieving Results for Index 5: quzhou, CN.
Closest hotel is Qufu Guandi Hotel.
------------
Retrieving Results for Index 6: panzhihua, CN.
Closest hotel is Shensen Hotel.
------------
Retrieving Results for Index 7: tabas, IR.
Closest hotel is Aastan Ghods Inn.
------------
Retrieving Results for Index 8: moree, AU.
Closest hotel is Albert Motel.
------------
Retrieving Results for Index 9: kombissiri, BF.
Closest hotel is Mitibkieta Annexe.
------------
Retrieving Results for Index 10:

C:\Users\ivanv\AppData\Local\Temp\ipykernel_22716\1598239952.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_names


In [40]:
missing_hotels = hotel_df.loc[hotel_df['Hotel Name'] == 'N/A']
missing_hotels.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,odweyne,SO,9.4092,45.0640,42,N/A
21,ndioum,SN,16.5129,-14.6471,16,N/A


In [42]:
hotel_df.drop(index = [20,21],inplace=True)

C:\Users\ivanv\AppData\Local\Temp\ipykernel_22716\1859119789.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.drop(index = [20,21],inplace=True)


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
fig2 = gmaps.figure()
humidity2 = hotel_df['Humidity']

heat_layer2 = gmaps.heatmap_layer(locations,
                                 weights=humidity2,
                                 dissipating=False,
                                 max_intensity=10,
                                 point_radius=1)

fig2.add_layer(heat_layer)
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
# Add the layer to the map
fig2.add_layer(markers)


# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))